In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')
from statsmodels.tsa.stattools import grangercausalitytests

# Exploratory analysis looking to examine the relationship between Marketing Budget and Repair Orders
# VMLYDJLSKJFDLKSJFDL...&R
# Alejandro Pineda
# 3/10/23


In [ ]:
# DEFINE FUNCTIONS UP TOP: IT'S CALLED 'FORESHADOWING'
def top_fiddy(df):
    
    keepr_columns = df.select_dtypes(include=['float64']).columns
    parts  = df[keepr_columns]
    #parts.drop('id0d92_svc_tot_a', inplace=True, axis=1)
    parts_sum = parts.sum()
    
    top_50 = parts_sum.sort_values(ascending=False)[0:50]
    return(top_50)

def pattern_find(columns, pattern=str):
    
   # Looks for columns that match a specific pattern

    reg_pattern = ".*" + pattern
    display(f"Looking for the following pattern: {reg_pattern}")
    
    r = re.compile(reg_pattern)
    col_list = list(filter(r.match, columns)) 
    display(col_list[0:5])
    return col_list

def explore_model(model):    
    print(model)
    
    f_path = 'C:/Users/Alejandro.Pineda/Documents/code/' + model
    print(f_path)
    
    if os.path.isdir(f_path) == False:
        os.mkdir(f_path)
    
    # temporary dataframe
    temp_df = romart22_ford[romart22_ford.id0d92_vmdl_desc_x == model]
   
    
    # histogram of repair time    
    plot1 = temp_df['repair_time'].dt.days.hist(bins = [10,20,30,40,50,60,70,80,90,100])
    plt.title(model + ': Repair Time (Days)')
    f_name1 = model + '/repair_time.png'
    plt.savefig(f_name1)
    
    # repair totals line plot
    plot2 = temp_df.plot('id0d92_svcs_open_y', 'id0d92_svc_tot_a')
    plt.title(model + ': Service Totals')
    f_name2 = model + '/repair_totals.png'
    plt.savefig(f_name2)
    
   
    #plot3 = plot_and_save(romart_dat2022, top_five_parts, 'id0d92_svcs_open_y')
    #f_name3 = model + '/top_parts.png'
    #plot3.savefig(f_name3)
   
    return plot1, plot2

    

#top_fiddy(temp_df)

In [ ]:
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [2]:

# LOAD THE REPAIR ORDER DATA
file = 'C:/Users/Alejandro.Pineda/Documents/data/ROMART_100K_sample.csv'
dat_romart = pd.read_csv(file)
display(dat_romart.shape)
dat_romart.head(50)
# look for vin to join / merge
# use start and end dates to capture length of repair (if we dont have it?)

(100000, 348)

,id0d92_ro_k,id0d92_vin_id_c,id0d92_repr_ordr_x,id0d92_svcs_open_y,id0d92_ro_closed_y,id0d92_cntry_c,id0d92_udb_srv_pa_c,id0d92_udb_pa_sub_c,id0d92_make_n,id0d92_vmdl_desc_x,...,id0d92_lamps_bulbs_prt_a,id0d92_starter_prt_f,id0d92_starter_prt_t,id0d92_starter_prt_a,id0d92_steering_susp_prt_f,id0d92_steering_susp_prt_t,id0d92_steering_susp_prt_a,id0d92_load_s,id0d92_lst_updt_usr_c,id0d92_lst_updt_s
0,00008 USA286017 2022/05/31,1ZVBP8CH3A5141554,286017,2022-05-31,2022-06-08,USA,00008,NaN,FORD,MUSTANG,...,0.00,0,0.0,0.0,0,0.0,0.0,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
1,00014 USA391973 2022/06/07,1FADP5BU6DL546032,391973,2022-06-07,2022-06-08,USA,00014,NaN,FORD,C-MAX,...,0.00,0,0.0,0.0,0,0.0,0.0,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
2,00018 USA00000000086151052022/02/08,1FBAX2CM2GKB24817,0000000008615105,2022-02-08,2022-06-08,USA,00018,NaN,FORD,TRANSIT,...,NaN,0,NaN,NaN,0,NaN,NaN,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
3,00025 USA316313 2022/06/08,1FTEW1EP0MKD97509,316313,2022-06-08,2022-06-09,USA,00025,NaN,FORD,F-150,...,0.00,0,0.0,0.0,0,0.0,0.0,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
4,00026 USA207197 2022/06/09,1FMCU9H68NUA43480,207197,2022-06-09,2022-06-09,USA,00026,NaN,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
5,00030 USA510327 2022/05/25,1FMCU0J98DUD02225,510327,2022-05-25,2022-06-08,USA,00030,NaN,FORD,ESCAPE,...,0.00,0,0.0,0.0,0,0.0,0.0,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
6,00031 USA097202 2022/05/19,2FMPK4J97JBC52904,097202,2022-05-19,2022-06-09,USA,00031,NaN,FORD,EDGE,...,NaN,0,NaN,NaN,0,NaN,NaN,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
7,00037 USA064260 2022/06/08,1FMCU9GX0FUC52840,064260,2022-06-08,2022-06-08,USA,00037,NaN,FORD,ESCAPE,...,0.00,0,0.0,0.0,0,0.0,0.0,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
8,00041 USA513002 2022/06/09,1FTEW1EPXMFC86037,513002,2022-06-09,2022-06-09,USA,00041,NaN,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39
9,00048 USA624763 2022/06/08,2FMPK3K97KBC52234,624763,2022-06-08,2022-06-08,USA,00048,NaN,FORD,EDGE,...,0.00,0,0.0,0.0,0,0.0,0.0,2022-06-12 18:35:39,ifpxhpam,2022-06-12 18:35:39


In [3]:
# LOAD THE MARKETING DATA

cm360_file = '/your/local/machine/.csv'
dat_cm360 = pd.read_csv(cm360_file)

dat_cm360.head()

#dat_cm360.groupby(['DMA']).count()

(52388, 92)

,AUDIENCE,PRIMARYTARGETING,VIDEODURATION,BRANDEDCONTENT,ISSPONSORSHIP,COBRANDEDFLAG,PACKAGETOTALBOOKEDUNITS,MEDIATYPEDESCRIPTION,ISPORTALHVPP,DMA,...,BOOKEDUNITS,CHANNEL,BONUSFLAG,INVESTMENTPRODUCTNAME,YOUTUBEVIDEOCATEGORY,ISVIDEO,RATETYPE,EXCLUDEFROMCHANNELREPORT,OBJECTIVE,_FIVETRAN_SYNCED
0,NaN,NaN,Unspecified,NaN,False,NaN,857143.0,NaN,False,NaN,...,0.0,NaN,NaN,unspecified,NaN,False,Flat Rate - Impressions,False,NaN,2022-10-03 08:08:16.184 +0000
1,NaN,NaN,Unspecified,NaN,False,NaN,227098.0,NaN,False,NaN,...,0.0,NaN,NaN,unspecified,NaN,False,Flat Rate - Impressions,False,NaN,2022-10-03 08:08:16.193 +0000
2,NaN,NaN,Unspecified,NaN,False,NaN,227098.0,NaN,False,NaN,...,0.0,NaN,NaN,unspecified,NaN,False,Flat Rate - Impressions,False,NaN,2022-10-03 08:08:16.167 +0000
3,NaN,NaN,Unspecified,NaN,False,NaN,397592.0,NaN,False,NaN,...,0.0,NaN,NaN,unspecified,NaN,False,Flat Rate - Impressions,False,NaN,2022-10-03 08:08:16.143 +0000
4,NaN,NaN,Unspecified,NaN,False,NaN,133858.0,NaN,False,NaN,...,0.0,NaN,NaN,unspecified,NaN,False,Flat Rate - Impressions,False,NaN,2022-10-03 08:08:16.154 +0000


In [ ]:
"""
MARCH ANALYSIS
# note: before running code on a model, specify the motivating question and why the model you're specifying
# answers the question... don't just run a model that works!

# Question: under what conditions does <spend> forecast <service total amounts>?
# Controlling for <warranty>, <make>, and <location>??

Possible vars for ARIMA multivariate model
Y:
<id0d92_svc_tot_a>: SERVICE TOTAL AMOUNT

X:
<id0d92_rpr_typ_wrnty_prts_a>: WARRANTY RPR TYPE PARTS AMOUNT
<id0d92_make_n>: VEHICLE MAKE NAME
<PLACEMENTTOTALPLANNEDMEDIACOST>: "spend"
<id0d92_dom_ma_c>: DOMINANT MARKETING AREA CODE

"""

In [ ]:
# DROP NULL VALUES (SUBSET FOR VARIABLES)
ro_vars_to_drop = ['vars', 'to', 'drop']

dat_romart = dat_romart.dropna(subset = ro_vars_to_drop)
dat_romart.shape

cm360_vars_to_drop = ['vars', 'to', 'drop']

dat_cm360 = dat_cm360.dropna(subset=cm360_vars_to_drop)
dat_cm360.shape

In [ ]:
# working with time variables

dat_romart['id0d92_svcs_open_y'] = pd.to_datetime(dat_romart['id0d92_svcs_open_y'])
dat_romart['id0d92_ro_closed_y'] = pd.to_datetime(dat_romart['id0d92_ro_closed_y'])
dat_romart['id0d92_load_s'] = pd.to_datetime(dat_romart['id0d92_load_s'])
dat_romart['id0d92_lst_updt_s'] = pd.to_datetime(dat_romart['id0d92_lst_updt_s'])



In [ ]:
# creating the repair_time variable
# dead end (3/10/23)
"""
dat_romart['repair_time'] = dat_romart['id0d92_ro_closed_y'] - dat_romart['id0d92_svcs_open_y']
display(dat_romart['repair_time'].head())

dat_cm360['STARTDATE'] = pd.to_datetime(dat_cm360['STARTDATE'])
dat_cm360.dropna(subset = ['STARTDATE'], inplace=True)
"""


In [ ]:
full_cols_romart = list(dat_romart.columns)
full_cols_romart

In [ ]:
dat_romart.dtypes

In [ ]:
# making sure time intervals match aross data sets

romart_range = str(dat_romart['id0d92_svcs_open_y'].dt.date.min()) + ' to ' +str(dat_romart['id0d92_svcs_open_y'].dt.date.max())
display("RO MART range is:", romart_range)

cm360_range = str(dat_cm360['STARTDATE'].dt.date.min()) + ' to ' +str(dat_cm360['STARTDATE'].dt.date.max())
display("MARKETING range is:", cm360_range)

In [ ]:
# LINE PLOT? 
# X VALUE: id0d92_svcs_open_y (dat_romart)
# Y VALUE 2 (repair orders):  id0d92_svc_tot_a (dat_romart)
# Y VALUE 1 (marketing dollars): PLACEMENTTOTALPLANNEDMEDIACOST


#plt.subplots(1,1,figsize=(16,9), dpi= 80)

romart_dat2022 = dat_romart[dat_romart['id0d92_svcs_open_y'] > '2022-02-01']
cmdat_2022 = dat_cm360[(dat_cm360['STARTDATE'] > '2022-02-01') & (dat_cm360['STARTDATE'] <= '2022-07-25')]

romart_dat2022 = romart_dat2022.sort_values(['id0d92_svcs_open_y'], ascending=[True])
cmdat_2022 = cmdat_2022.sort_values(['STARTDATE'], ascending=[True])
# define x and y's




x1 = romart_dat2022['id0d92_svcs_open_y']
y1 = romart_dat2022['id0d92_lamps_bulbs_prt_a']

x2 = romart_dat2022['id0d92_svcs_open_y']
y2 = romart_dat2022['id0d92_tire_prt_a']

# Creates two axes, and accesses them through the returned array

fig, axes = plt.subplots(2, 1,figsize=(9,6), dpi= 80, sharex = 'col')
axes[0].plot(x1, y1, '--')
axes[0].set_title('Lamp Bulbs')
axes[1].plot(x2, y2, '--')
axes[1].set_title('Tires')
plt.show()





In [ ]:
# making sure time intervals match aross data sets

romart_range = str(romart_dat2022['id0d92_svcs_open_y'].dt.date.min()) + ' to ' +str(romart_dat2022['id0d92_svcs_open_y'].dt.date.max())
display("RO MART range is:", romart_range)

cm360_range = str(cmdat_2022['STARTDATE'].dt.date.min()) + ' to ' +str(cmdat_2022['STARTDATE'].dt.date.max())
display("MARKETING range is:", cm360_range)

In [ ]:
n_cmdat = len(cmdat_2022)
n_cmdat

In [ ]:
# make sure dimensions match for Granger Analysis below

romart_dat2022_small = romart_dat2022.sample(n=n_cmdat, random_state=1234)
romart_dat2022_small = romart_dat2022_small.sort_values(['id0d92_svcs_open_y'], ascending=[True])

cmdat_2022 = cmdat_2022.sort_values(['STARTDATE'], ascending=[True])

# define x and y's
romart_dat2022_small.shape



In [ ]:
romart_dat2022_small['id0d92_svc_tot_a'].hist(bins=10, range=[0, 1000])

In [ ]:
# 

merged_data = romart_dat2022_small[['id0d92_svc_tot_a','id0d92_rpr_typ_wrnty_prts_a','id0d92_dom_ma_c','id0d92_make_n']]
merged_data['spend'] = cmdat_2022[['PLACEMENTTOTALPLANNEDMEDIACOST']]
merged_data.head()

merged_data[['spend']].isna().sum()

In [ ]:
merged_data_n = merged_data.shape[0]
test_n = int(.2 * merged_data_n)
train_n = int(merged_data_n - test_n)


In [ ]:
train_n

In [ ]:
# Dealing with NaN's
merged_data.replace([np.inf, -np.inf], np.nan, inplace=True)
merged_data = merged_data.fillna(method='bfill', limit = 25)
merged_data = merged_data.dropna(axis='columns', thresh=.75)

In [ ]:
merged_data.describe()

In [ ]:
# Dealing with Float64 variables (too big for the regression)

# Select columns with 'float64' dtype  
float64_cols = list(merged_data.select_dtypes(include='float64'))

# The same code again calling the columns
merged_data[float64_cols] = merged_data[float64_cols].astype('float16')



In [ ]:
y = merged_data[['id0d92_svc_tot_a']]
X = merged_data[keeper_vars]

In [ ]:
time_plot = merged_data.plot.line(x='id0d92_svcs_open_y')


In [ ]:
# Split the data into training/testing sets
merged_X_train = X[:-test_n]
merged_X_test = X[-test_n:]

# Split the targets into training/testing sets
merged_y_train = y[:-test_n]
merged_y_test = y[-test_n:]

In [ ]:
merged_y_test.shape

In [ ]:
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(merged_X_train, merged_y_train)

# Make predictions using the testing set
merged_y_pred = regr.predict(merged_X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(merged_y_test, merged_y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(merged_y_test, merged_y_pred))



In [ ]:
fcsd_X_train

In [ ]:
from mpl_toolkits import mplot3d
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Creating plot
sctt = ax.scatter3D(fcsd_X_test[['MEDIACOST']].values, fcsd_X_test[['DARTCOST']].values, fcsd_y_test.values, color = "green")
plt.title("Costs versus Impressions")
ax.set_xlabel('Media', fontweight ='bold')
ax.set_ylabel('Dart', fontweight ='bold')
ax.set_zlabel('US Impressions', fontweight ='bold')
plt.axis([0, 15, 0, 15])
# show plot
plt.show()

In [ ]:
plt.plot(fcsd_X_test, fcsd_y_pred, color="blue", linewidth=3)
plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
name = fcsd_X_train['car'].head(12)
price = fcsd_X_train['price'].head(12)
 
# Figure Size
fig, ax = plt.subplots(figsize =(16, 9))
 
# Horizontal Bar Plot
ax.barh(name, price)
 
# Remove axes splines
for s in ['top', 'bottom', 'left', 'right']:
    ax.spines[s].set_visible(False)
 
# Remove x, y Ticks
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')
 
# Add padding between axes and labels
ax.xaxis.set_tick_params(pad = 5)
ax.yaxis.set_tick_params(pad = 10)
 
# Add x, y gridlines
ax.grid(b = True, color ='grey',
        linestyle ='-.', linewidth = 0.5,
        alpha = 0.2)
 
# Show top values
ax.invert_yaxis()
 
# Add annotation to bars
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize = 10, fontweight ='bold',
             color ='grey')
 
# Add Plot Title
ax.set_title('Sports car and their price in crore',
             loc ='left', )
 
# Add Text watermark
fig.text(0.9, 0.15, 'Jeeteshgavande30', fontsize = 12,
         color ='grey', ha ='right', va ='bottom',
         alpha = 0.7)
 
# Show Plot
plt.show()